# Canadian Social Harmony Index Visualization Project

[1. Data Exploration](#section1) 


In [2]:
# import all the required packages
import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.express as px

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

<a id="section1"></a>

## 1. Data Exploration

#### Read .csv file

In [3]:
df_responses = pd.read_csv('./data/dfg_survey_responses.csv')
df_values = pd.read_csv('./data/dfg_values_key-question-key.csv')
df_scale = pd.read_csv('./data/sixPointScale.csv')

In [4]:
df_responses['WT'].sum()

1575.0

In [5]:
df_responses['WT'].shape[0]

1575

In [11]:
df_responses.head(5)

,ID,WT,FSA,Rural_FSA,Region,YOB,Age,Gender,Education,Children,Religion,Politics_Spectrum,Born_Current_City,Born_Current_Province,Born_Current_Country,Born_Parents,Income,Financial_Security,Financial_400,Employment,Occupation_Type,Home_Ownership,SV_01,SV_02,SV_03,SV_04,SV_05,SV_06,SV_07,SV_08,SV_09,SV_10,SV_11,SV_12,SV_13,SV_14,SV_15,SV_16,SV_17,SV_18,SV_19,SV_20,Conflict_1,Conflict_2,Conflict_3
0,38,0.441838,S4X,No,Prairies,1952,55+,Male,Bachelors,Yes,Not important,Progressive/liberal views,NaN,NaN,NaN,All parents born in Canada,"$150,000 - $199,999",Yes,Put it on my credit card and pay it off in ful...,Unemployed,"Occupations in art, culture, recreation and sport",Owned by you or a member of your household (ev...,A little like me,Not like me,Somewhat like me,Like me,Somewhat like me,Not like me,A little like me,Not like me,A little like me,Somewhat like me,Somewhat like me,A little like me,Somewhat like me,Like me,A little like me,A little like me,Somewhat like me,Not like me,Not like me,A little like me,People who share my views need to be willing t...,"In Canada, our differences are not so large an...",I seek to find common understanding with other...
1,39,0.616034,R3E,No,Prairies,1989,18-34,Female,Advanced,Yes,Somewhat important,Progressive/liberal views,NaN,NaN,NaN,All parents born outside Canada,"0 - $34,999",Yes,Put it on my credit card and pay it off over time,Part-time,Health occupations,Rented (even if no cash rent is paid)?,A little like me,Somewhat like me,Somewhat like me,Like me,Very much like me,A little like me,Like me,Somewhat like me,Like me,Like me,A little like me,Somewhat like me,Like me,Somewhat like me,Somewhat like me,Somewhat like me,Somewhat like me,A little like me,Somewhat like me,Somewhat like me,People who share my views need to be willing t...,"In Canada, our differences are not so large an...",Some people’s beliefs are so unlike mine that ...
2,40,0.664991,M1B,No,Ontario,1959,55+,Male,Bachelors,Yes,Very important,Both conservative and liberal views,NaN,NaN,NaN,All parents born outside Canada,NaN,NaN,Prefer not to say,Out of workforce,Sales and service occupations,Rented (even if no cash rent is paid)?,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Somewhat like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,People who share my views should stick to thei...,"In Canada, our differences are not so large an...",I seek to find common understanding with other...
3,41,1.269547,V9V,No,BC,1958,55+,Female,HS,Yes,Somewhat important,Both conservative and liberal views,NaN,NaN,NaN,All parents born in Canada,"$70,000 - $99,999",Yes,Put it on my credit card and pay it off in ful...,Full-time,"Business, finance and administration occupations",Owned by you or a member of your household (ev...,Somewhat like me,Like me,A little like me,Like me,Somewhat like me,Somewhat like me,Somewhat like me,A little like me,Somewhat like me,Very much like me,Somewhat like me,A little like me,Somewhat like me,A little like me,Somewhat like me,A little like me,Somewhat like me,Not like me,Not like me,Somewhat like me,People who share my views need to be willing t...,"In Canada, our differences are not so large an...",I seek to find common understanding with other...
4,42,0.518667,V6Y,No,BC,1979,35-54,Male,Bachelors,Yes,Somewhat important,Both conservative and liberal views,NaN,NaN,NaN,All parents born outside Canada,"$70,000 - $99,999",Yes,Put it on my credit card and pay it off over time,Full-time,Management occupations,Owned by you or a member of your household (ev...,Very much like me,Not like me,Somewhat like me,Somewhat like me,Very much like me,Very much like me,Like me,Very much like me,A little like me,Somewhat like me,Not like me,Very much like me,Somewhat like me,Like me,Li

In [42]:
df_values

,Key,Value,Question,Question Group,Order
0,SV_01,Conformity,They believe they should always show respect t...,Conservation,5
1,SV_02,Tradition,Religious belief is important to them. They tr...,Conservation,6
2,SV_03,Benevolence,It's very important to them to help the people...,Relationships,4
3,SV_04,Universalism,They think it is important that every person i...,Relationships,3
4,SV_05,Selfdirection,They think it's important to be interested in ...,Independence,12
5,SV_06,Stimulation,They like to take risks. They are always looki...,Independence,11
6,SV_07,Hedonism,They seek every chance they can to have fun. I...,Independence,10
7,SV_08,Achievement,Getting ahead in life is important to them. Th...,Self-Enchantment,9
8,SV_09,Power,They always want to be the one who makes the d...,Self-Enchantment,8
9,SV_10,Security,It is important to them that things be organiz...,Conservation,7


In [7]:
df_scale

,Rating,Sort like to not like,Scale order
0,Not like me at all,6,1
1,Not like me,5,2
2,Somewhat like me,4,3
3,A little like me,3,4
4,Like me,2,5
5,Very much like me,1,6


## 1.1 Widget
#### - Explore each column

In [38]:
@interact
def bar_chart(col_name=list(df_responses.columns)):
    if (col_name=="ID"):
        fig = px.scatter(df_responses[col_name])
    else:
        df = df_responses.groupby(col_name).agg({'WT': ['sum','count']}).reset_index()
        df.columns = df.columns.map(''.join)
        df.rename(columns={'WTsum': 'Total_Wt', 'WTcount': 'Total'}, inplace = True)
        fig  = px.bar(df, x=col_name, y = ['Total', 'Total_Wt'], barmode='group')
    fig.show()

interactive(children=(Dropdown(description='col_name', options=('ID', 'WT', 'FSA', 'Rural_FSA', 'Region', 'YOB…

In [39]:
@interact
def bar_chart(col_name=list(df_values.columns)):
    fig = px.bar(df_values.groupby(col_name)[col_name].count())
    fig.show()

interactive(children=(Dropdown(description='col_name', options=('Key', 'Value', 'Question', 'Question Group', …

# Ideas worth exploring
1. Merge two data frames and find scores by the 5 categories for each question.
2. How do we find the majority of Canadians think about it and take the median? 
3. There is not much connection between the survey questions and the outcomes?  Project input variables apart from the SV questions and see if there is any connection.
5. Plot on the boundary files and project the input and results. Looking province by province. 